<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-description" data-toc-modified-id="Project-description-1">Project description</a></span></li><li><span><a href="#Project-plan:" data-toc-modified-id="Project-plan:-2">Project plan:</a></span></li><li><span><a href="#Data-description" data-toc-modified-id="Data-description-3">Data description</a></span></li><li><span><a href="#Read-data" data-toc-modified-id="Read-data-4">Read data</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-5">Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-6">Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-7">Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-8">Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-9">Чек-лист проверки</a></span></li></ul></div>

# Protection of insurance company's customer data

## Project description
It is necessary to protect the customer data of the insurance company.

The task: develop a data conversion method to make it difficult to recover personal information after conversion. Prove the correctness of it's work. 

Need to protect the data without reducing the quality of machine learning model after conversion. It is not necessary to select the best model.

## Project plan:
- 



## Data description
__Features:__ gender, age and salary of the insured, the number of his family members.

__Target feature__: the number of insurance payments to the client over the last 5 years.

## Read data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
display(df.head())
df.info()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


There are no missing values, the column values are numerical.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится.

**Обоснование:** Для начала посмотрим, как изменится вектор весов линейной регрессии $w$ при умножении признаков на обратимую матрицу.

Веса "до":

$
w = (X^T X)^{-1} X^T y
$

Веса "после":

$
w_P = ((XP)^T (XP))^{-1} (XP)^T y
$

Преобразуем выражение "после", чтобы привести его к виду, более удобному для сравнения с "до".

***Свойство о матрицах №1***: Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке. Преобразуем участок:

$
(XP)^T = P^TX^T
$

И получим:

$
((XP)^T (XP))^{-1} (XP)^T y = ((P^TX^T)(XP))^{-1} (P^TX^T) y
$

***Свойство о матрицах №2***: Ассоциативность умножения:

$A(BC) = (AB)C$

Преобразуем участок:

$
(P^TX^T)(XP) = (P^TX^TX)P
$

И получим:

$
((P^TX^T)(XP))^{-1} (P^TX^T) y = ((P^TX^TX)P)^{-1} P^TX^T y
$

***Свойство о матрицах №3***: Произведение обратимых матриц есть обратимая матрица. $P$ и $X^TX$ обратимы по условию. $P^T$ тоже обратима по еще одному свойству матриц. То есть $P^TX^TX$ и $P$ обратимые матрицы.

***Свойство о матрицах №4***: Если $A$ и $B$ обратимые матрицы, то:

$
(AB)^{-1} = B^{-1}A^{-1}
$

Преобразуем участок:

$
((P^TX^TX)P)^{-1} = P^{-1}(P^TX^TX)^{-1} = P^{-1}(X^TX)^{-1}(P^T)^{-1}
$

И получим:

$
((P^TX^TX)P)^{-1} P^TX^T y = P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^T y
$

***Свойство о матрицах №5***: Умножение матрицы на обратную равно единичной матрице:

Преобразуем:

$
(P^T)^{-1} P^T = E
$

Получим:

$
P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^T y = P^{-1}(X^TX)^{-1}EX^T y
$

***Свойство о матрицах №6***: Произведение матрицы на единичную матрицу равняется первоначальной матрице:

$
P^{-1}(X^TX)^{-1}EX^T y = P^{-1}(X^TX)^{-1}X^T y
$

***В итоге получили***:

$
w_P = P^{-1}(X^TX)^{-1}X^T y
$

Помним, что изначально параметр линейной регрессии:

$
w = (X^TX)^{-1}X^T y
$

Следовательно, при умножении признаков на обратимую матрицу $P$ параметр линейной регрессии умножается на обратную матрицу $P^{-1}$.

$
w_P = P^{-1}w
$

Подставим параметры в формулы предсказания до и после умножения признаков на обратимую матрицу.

До:

$
a = Xw
$

После:

$
a_P = (XP)w_P = (XP)(P^{-1}w) = XPP^{-1}w = XEw = Xw
$

Получили равенство $a = a_P$.

Качество линейной регрессии не изменится, потому что значения предсказаний при переобучении модели не изменится.

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>



Успех 👍:




Самая частая ошибка - не учитывают что обратная матрица может быть только у квадратной.  Часто не соблюдают правила матричного преобразования, или меняют матрицы местами при перемножении как цифры.  


У тебя все ОК. 
    





    
    
<div class="alert alert-warning">



Совет 🤔:    
    
    
  
Есть ещё один способ доказательства   когда мы берём
    
    
$$
w = \arg\min_w MSE(Xw, y)
$$
    
  
и     
    
$$
w_P = \arg\min_w MSE(XPw_p, y)
$$  
    
    
И делая  замену    w_p, видим что задача  
    
$$
\arg\min_w MSE(XPw_p, y)
$$
    
сводится к     
  
    
$$
\arg\min_w MSE(Xw, y)
$$    


Осталось только догадаться какую замену мы делаем.  Можешь попробовать решить и таким способом  

  
</div>
    

    





<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка ❌:
    


  
Нет ответа на "как связаны параметры линейной регрессии в исходной задаче и в преобразованной"

Какое соотношение между $w$ и $w_p$?
    
    
Где   



$$
w = (X^T X)^{-1} X^T y
$$
    
    
$$
w_P = ((XP)^T XP)^{-1} (XP)^T y
$$   


<div class="alert" style="background-color:#ffd65a"> <b>Комментарий студента:</b> Исправила. Немного изменила обоснование, почему качество линейной регрессии не изменится. Сначала вывела соотношение между $w$ и $w_p$, а потом подставила их в формулу предсказания и доказала равенство предзказаний. Старая версия в ячейке ниже.</div>

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Успех 👍:



Привет Мария! Заждался я тебя ) Соотношение верное




</div>


Старая версия доказателства:
<details>
    **Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

    **Ответ:** Не изменится.

    **Обоснование:** Сравним формулы предсказания до и после умножения признаков на обратимую матрицу.

    До:

    $
    a = Xw = X (X^T X)^{-1} X^T y
    $

    После:

    $
    a = (XP)w = XP ((XP)^T (XP))^{-1} (XP)^T y
    $

    Преобразуем выражение "После", чтобы доказать его равенство "До".

    ***Свойство о матрицах №1***: Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке. Преобразуем участок:

    $
    (XP)^T = P^TX^T
    $

    И получим:

    $
    XP ((XP)^T (XP))^{-1} (XP)^T y = XP ((P^TX^T)(XP))^{-1} (P^TX^T) y
    $

    ***Свойство о матрицах №2***: Ассоциативность умножения:

    $A(BC) = (AB)C$

    Преобразуем участок:

    $
    (P^TX^T)(XP) = (P^TX^TX)P
    $

    И получим:

    $
    XP ((P^TX^T)(XP))^{-1} (P^TX^T) y = XP ((P^TX^TX)P)^{-1} P^TX^T y
    $

    ***Свойство о матрицах №3***: Произведение обратимых матриц есть обратимая матрица. $P$ и $X^TX$ обратимы по условию. $P^T$ тоже обратима по еще одному свойству матриц. То есть $P^TX^TX$ и $P$ обратимые матрицы.

    ***Свойство о матрицах №4***: Если $A$ и $B$ обратимые матрицы, то:

    $
    (AB)^{-1} = B^{-1}A^{-1}
    $

    Преобразуем участок:

    $
    ((P^TX^TX)P)^{-1} = P^{-1}(P^TX^TX)^{-1} = P^{-1}(X^TX)^{-1}(P^T)^{-1}
    $

    И получим:

    $
    XP ((P^TX^TX)P)^{-1} P^TX^T y = XP P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^T y
    $

    ***Свойство о матрицах №5***: Умножение матрицы на обратную равно единичной матрице:

    Преобразуем:

    $
    P P^{-1} = E
    $

    $
    (P^T)^{-1} P^T = E
    $

    Получим:

    $
    XP P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^T y = XE(X^TX)^{-1}EX^T y
    $

    ***Свойство о матрицах №6***: Произведение матрицы на единичную матрицу равняется первоначальной матрице:

    $
    XE(X^TX)^{-1}EX^T y = X(X^TX)^{-1}X^T y
    $


    Получили равенство $Xw = (XP)w$, доказано.

    Качество линейной регрессии не изменится, потому что значения предсказаний при переобучении модели не изменится.
</details>

## Алгоритм преобразования

**Алгоритм**

1. Отделить признаки от целевого признака;
2. Сгенерировать случайную обратимую матрицу. Она будет квадратной (это условие обратимости матрицы) с размерностью, равной количеству признаков;
3. Умножить признаки на обратимую матрицу. Размерность обратимой матрицы подобрана так, что размерность матрицы признаков после преобразования не изменится.


<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех 👍:


Принято

<div class="alert alert-warning">

Совет 🤔:

А если сгенерированная матрица будет необратимой? Тут можно расписать порядок действий на все случаи в том числе на этот. Кроме того можно написать как выбираем размер генерируемой матрицы
</div>


<div class="alert" style="background-color:#ffd65a"> <b>Комментарий студента:</b> Добавила объяснение выбранного размера генерируемой матрицы. Объяснения на случай, если сгенерированная матрица будет не обратимой, не добавляла, так как далее исключила риск, что матрица может создаться необратимой.</div>

<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюераV2</b></font>



Совет 🤔:



Так в алгоритме стоило прописать логику с помощью которой ты будешь гарантировать обратимость матрицы:
    
    
    
...
    
    
3. Генерируем матрицу такого-то размера
    
    
4. Проверяем Матрицу на обратимость
    
    
    
5. Если Матрица необратимая возвращаемся в пункт 3    

**Обоснование**

При переобучении модели на преобразованных признаках качество линейной регрессии не изменится, потому что не изменится значения предсказаний. Доказано в пункте 2.

## Проверка алгоритма

Разделим данные на обучающую и валидационную выборки.

In [3]:
df_features = df.drop(['Страховые выплаты'], axis=1)
df_target = df['Страховые выплаты']

features_train, features_test, target_train, target_test = train_test_split(df_features,
                                                                            df_target,
                                                                            train_size=0.25,
                                                                            random_state=12345)

Обучим модель и посчитаем метрику R2.

In [4]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)

display(predictions)
print('R2 =', r2_score(target_test, predictions))

array([ 0.1748403 ,  0.77684954,  0.432202  , ...,  0.63476958,
       -0.14363463, -0.13233092])

R2 = 0.4256512934658


Теперь проверим, как изменятся предсказания и метрика R2 после применения алгоритма преобразования.

In [5]:
# create random invertible matrix

def get_rand_inv_matrix(n):
    det = 0
    np.random.seed(123)
    while det == 0:
        matrix = np.random.normal(size=(n, n))
        det = np.linalg.det(matrix)
    return matrix




<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюера</b></font>

Совет 🤔:


- ты проверяешь    матрицу на обратимость - отлично, но если она необратимая выскочит ошибка, код не исполнится до конца
 

Можно здесь было сделать похитрей - генерируем матрицу пока не получим обратимую и наш код  гарантированно полностью отработает. Добиться этого можно например так:


    
    def get_rand_matrix():
        det = 0
        while det == 0:
            matrix = np.random.normal(size=(data.shape[1], data.shape[1]))
            det = np.linalg.det(matrix)
        return matrix

C помощью рекурсии

    def func(dim):
        transform_matrix = np.random.rand(dim, dim)
        try: 
            inv = np.linalg.inv(transform_matrix)
            return transform_matrix, inv
        except:
            func(dim)  
    

<div class="alert" style="background-color:#ffd65a"> <b>Комментарий студента:</b> Добавила.</div>

In [6]:
def encode_features(df_features):
    matrix = get_rand_inv_matrix(df_features.shape[1])
    return pd.DataFrame((df_features.values@matrix), columns = df_features.columns)

encode_pipeline = Pipeline(steps=[
        ('encode_features', FunctionTransformer(encode_features)),
        ('model', LinearRegression())
])

In [7]:
encode_pipeline.fit(features_train, target_train)
predictions_on_encoded_data = encode_pipeline.predict(features_test)

display(predictions_on_encoded_data)
print(r2_score(target_test, predictions_on_encoded_data))

array([ 0.1748403 ,  0.77684954,  0.432202  , ...,  0.63476958,
       -0.14363463, -0.13233092])

0.42565129346573494


Метрика R2 и значения предсказаний не изменились после преобразований. Значит, можно применять этот алгоритм защиты данных без ухудшения качества линейной регрессии.


## Вывод ##

В рамках проектной работы:
1. Изучены данные.
2. Доказано, что при умножении матрицы признаков на обратимую матрицу, качество линейной регрессии не изменится.
3. Выведено соотношение параметров линейной регрессии до и после умножения признаков на обратимую матрицу.
4. Запрограммирован алгоритм преобразования данных на основе умножения матрицы признаков на обратимую матрицу.
5. На практике проверено, что качество линейной регрессии не меняется после применения алгоритма.



<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех 👍:
    
Все сошлось. Отдельный плюс за: 
    
    
- универсальность кода: код отработает для любой размерности матрицы обьект-признак  



- использование train_test_split - все правильно, помним о стандартах ML при моделировании

    
    

<div class="alert alert-warning">


Совет 🤔: 
    


- можно реализовать pipeline, с собственной функцией преобразования данных. Это может быть полезно если ты уже использовал pipeline со стандартными функциями sklearn (Scaler,MInMax итд итп), но хотел  бы попробовать создать свою функцию преобразования - в нашем случае перемножение наших данные на сгенерированную матрицу. Какого то практического смысла это иметь не будет, напомни что pipeline мы используем для того чтобы не было утечки данных (плюс достигается краткость ), зато можно поупражняться.

 
Можешь взять за основу [Ссылка 1](https://dzen.ru/media/id/5ee6f73b7cadb75a66e4c7e3/sozdanie-polzovatelskih-preobrazovatelei-dannyh-62b2a9a80e49941961ffc7a2),
[Ссылка 2](https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156)

На выходе мы должны получить что-то вроде

    pipe_cripto = Pipeline([
            ('transformer', MatrixTransformer(X.columns)),
            ('model', LinearRegression()),
        ])


    pipe_cripto.fit(X_train)

    pipe_cripto.predict(X_test)



<div class="alert" style="background-color:#ffd65a"> <b>Комментарий студента:</b> Добавила. В первый раз использую pipeline, буду рада замечаниям. </div>

<div class="alert alert-warning">
<font size="5"><b>Комментарий ревьюера</b></font>

Совет 🤔:



Можно ещё через создание класс, тогда будет выглядеть так
    
Схема:
    


    
1. Создаем класс    
    
    
    
    class MatrixTransformer(BaseEstimator, TransformerMixin):

        def __init__(self, variables):
            """В конструкторе сохраним имена колонок и сгененрируем матрицу преобразования"""
            self.variables = variables
            size = len(variables)
            rng = np.random.default_rng(seed=RANDOM_STATE)
            self.M = rng.normal(size=(size, size))

        def fit(self, X, y):
            return self

        def transform(self, X):
            X_tr = X.drop(self.variables, axis=1)
            X_tr[self.variables] = X[self.variables] @ self.M
            return X_tr


2. Иcпольpуем pipeline
    
    
    

    pipe = Pipeline([
        ('transformer', MatrixTransformer(X.columns)),
        ('model', LinearRegression()),
    ])

        pipe.fit(X_train, y_train)
    
        pipe.predict(X_test, y_test)


    
    




<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка ❌:

Мария, было бы замечательно, в конце немножко о данных, работе с ними, моделировании и полученных результатах, доказательствах. Это важно потому что когда проект захочет посмотреть будущий работодатель (или начальник), у него может не быть времени на подробный разбор кода. Вероятнее всего он бегло просмотрит код, а из общего вывода захочет получить представление о всей работе. Вырабатывай рефлексы правильного оформления проектов )


</div>

<div class="alert" style="background-color:#ffd65a"> <b>Комментарий студента:</b> Добавила вывод.</div>


<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюера</b></font>


Мария, у тебя старательно выполненная работа, все четко, осмысленно. Отмечаю уровень твоей программистской подготовки. 



    

Я оставил небольшие советы и вопросики (если есть время и желание можешь воспользоваться/ответить)
    


Обязательное к исправлению:





- выводим соотношение между  𝑤  и  𝑤𝑝 




    
- стоит добавить общий вывод (лицо проекта)


 




Жду исправлений, для принятия проекта. Если какие то вопросы, то сразу спрашивай ) 








<div class="alert alert-warning">
<b>Комментарий ревьюера</b>



На мой взгляд проект очень необычный, выделяющийся из остальных проектов потому что касается раздела математики лежащей в основе машинного обучения  (помимо производных, матанализа и теории вероятности с матстатистикой). Если в дальнейшем ты углубишься в изучении нейростетей, то понимание линейной алгебры будет очень полезным, потому что, по сути, все нейросети - это перемножение матриц и Линейная алгебра даёт очень компактный способ записи того что происходит в нейросетях. 
    
    
    
Возможно будет полезно:
    
    
- Теория по обратным матрицам:  [тут](https://www.berdov.com/works/matrix/obratnaya-matrica/)

    
- Если хочешь посмотреть вообще про линейную алгебру и матрицы, то мне нравится  вот это [видео](https://www.youtube.com/watch?v=RNTRYicPvWQ&list=PLVjLpKXnAGLXPaS7FRBjd5yZeXwJxZil2), рассказывается по существу и увлекательно.  


- по применению криптографии в машинном обучении на английском: [тут](https://www.cs.cmu.edu/~rjhall/JOS_revised_May_31a.pdf)




<div class="alert" style="background-color:#ffd65a"> <b>Комментарий студента:</b> Спасибо за замечания! </div>


<div class="alert alert-info">
<font size="5"><b>Комментарий ревьюераV2</b></font>

Спасибо за работу!    

    
Красное исправлено,  желтые советы использованы. Надеюсь мои советы и вопросики были полезны и в копилочку знаний упало что то новое, а проект стал лучше, и симпатичней.


  
Отличная работа Мария. Желаю успехов в дальнейшей учебе!


## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования